In [1]:
from dotenv import load_dotenv
import os
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv(dotenv_path='../.env')

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])

gs


In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser

from pydantic import BaseModel, Field
from typing import List
from pprint import pprint

In [10]:
class IntroduceMySelf(BaseModel):
    name: str = Field(description="이름")
    age: str = Field(description="나이")
    major : str = Field(description="전공")
    hobbies: List[str] = Field(description="취미")
    goal: str = Field(description="목표")


In [14]:
def introduce_keywords(letter):
    parser = PydanticOutputParser(pydantic_object=IntroduceMySelf)
    template = """
        다음 사용자의 자기소개에 따라 키워드를 분리해내세요
        요청: {query}
        {format_instructions}
    """

    prompt = ChatPromptTemplate.from_template(template=template)

    prompt = prompt.partial(
    format_instructions=parser.get_format_instructions()
    )
    
    model = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="moonshotai/kimi-k2-instruct-0905",
    temperature=0.7
    )

    
    chain = prompt | model | parser
    return (chain.invoke({"query": letter}))

    


In [21]:
introduce_keywords("안녕하세요 전 박시훈이고, 컴퓨터공학과이고요 27살이에요. 자전거타고 하이킹하는걸 좋아해요 다리를 다쳐서 지금은 못하지만요.. 지금은 유튜브 보거나 전자기기 없이 숲속을 걷는걸로 만족하고 있어요 sk 쉴더스에 입사하기 위해 노력하고 있어요")

IntroduceMySelf(name='박시훈', age='27', major='컴퓨터공학과', hobbies=['자전거', '하이킹', '유튜브 시청', '숲속 걷기'], goal='SK 쉴더스 입사')